## 6. 모델 선택과 훈련
주어진 데이터셋의 가공과 정제가 완료되었다면 올바른 모델을 선택하여 모델을 학습시키면 된다. 

### (1)Linear Regression
Linear Regression은 입력변수와 계수들의 Inner Product하여 일차식을 만들되 RMSE의 값이 최소가 되도록 하는 것이 목표이다. sklearn.linear_model의 LinearRegression을 이용해서 객체를 생성한 후 fit()메서드를 통해 훈련시킨 후 predict()메서드를 이용해서 새로운 입력변수에 대한 예측값을 얻을 수 있다.

In [37]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_label)

LinearRegression()

In [38]:
some_data = housing.iloc[:5]
some_labels = housing_label.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predicted : ", lin_reg.predict(some_data_prepared))
print("Labels : ", list(some_labels))

Predicted :  [ 88308.5135735  304174.12243205 154135.1578518  183826.73869117
 247307.37672157]
Labels :  [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


> ℹ️ **부가 설명 | RMSE측정**
sklearn.metrics의 mean_squared_error를 통해 mse를 구한후 numpy의 sqrt메서드를 이용해서 제곱근을 구해 RMSE를 구할 수 있다. 아래의 경우 중간 주택 가격은 $120000 ~ $265000이지만 RMSE가 68911이므로 성능이 그닥 좋지 못하다. 이는 특성들이 좋은 예측을 만들 만큼 충분한 정보를 제공하지 못했거나 모델이 충분히 강력하지 못하다는 사실을 말한다.

In [39]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_label, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

np.float64(68911.35422344579)

### (2) Decision Tree Regression
Decision Tree Regression은 target variable과의 variance를 가장 크게 만드는 변수들 기준으로 계속 branch를 나누어가며 하위노드를 만들어 Decision Tree를 생성한다. Predict()메서드를 이용해서 값을 예측하는 경우, 도달한 리프노드에 속하는 데이터들의 타겟변수 평균으로 값을 예측한다. 아래의 경우 RMSE가 0이 나오는 것으로 보아 과대적합된 것으로 보인다. (당연히 train_set전체를 이용해서 평가를 했으니 RMSE가 0에 수렴할 것이다.)

In [40]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_label)

DecisionTreeRegressor()

In [41]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_label, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

np.float64(0.0)

### (3)K - fold cross - validation
교차검증을 이용해서 훈련 데이터를 여러개의 폴드로 나누어 계속 훈련하고 평가할 수 있다. 이때 sklearn.model_selection의 cross_val_score을 사용하면 된다. 이때 scoring함수로는 "neg_mean_sqaured_error"를 사용해야하며 제곱근을 사용하기전에 부호를 바꿔야한다.

In [42]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_label, scoring = "neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores) 


In [43]:
def display_scores(scores):
    print("score : ", scores)
    print("average : ", scores.mean())
    print("standard : ", scores.std())
display_scores(tree_rmse_scores)

score :  [69585.58434462 69402.04142821 69139.73461339 72934.65353599
 66836.88306627 75997.19439137 71317.19121201 72068.09203668
 69567.24839393 70969.82254514]
average :  70781.84455676127
standard :  2384.9572318697524


In [44]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_label, scoring = "neg_mean_squared_error", cv = 10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

score :  [72327.16076767 64517.26490514 68064.68293242 69115.0013636
 67146.11690224 73027.74105799 70586.20149172 69341.53755437
 66928.91520262 70506.05866694]
average :  69156.06808447084
standard :  2462.9099081209392


### (4) Random Forest Regression
Random Forest Regression은 여러개의 Decision Tree를 데이터 복원추출을 통해 생성한 후 입력값을 생성된 모든 Decision Tree에 넣고 얻어진 모든 결과값들의 평균을 계산하여 출력한다. 이 때문에 시간복잡도가 증가한다.

In [45]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_label)
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_label, scoring = "neg_mean_squared_error", cv = 10)
forest_rmse =np.sqrt(-forest_scores)
display_scores(forest_rmse)

score :  [51214.76493248 49128.40348961 46645.8891357  52075.92327914
 47634.49828803 51144.78721284 52140.52684261 50276.754989
 48455.47240174 53870.63332855]
average :  50258.76538996907
standard :  2146.0943284104796
